In [149]:
import os
import struct
import numpy as np
import intvalpy as ip
import matplotlib.pyplot as plt
from functools import cmp_to_key
from pathlib import Path
from scipy.stats import linregress
from tqdm import tqdm

In [150]:
def read_bin_file_with_numpy(file_path):
  with open(file_path, 'rb') as f:
    header_data = f.read(256)
    side, mode, frame_count = struct.unpack('<BBH', header_data[:4])

    frames = []
    point_dtype = np.dtype('<8H')

    for _ in range(frame_count):
      frame_header_data = f.read(16)
      stop_point, timestamp = struct.unpack('<HL', frame_header_data[:6])
      frame_data = np.frombuffer(f.read(1024 * 16), dtype=point_dtype)
      frames.append(frame_data)

    return np.array(frames)

In [151]:
def convert_to_voltage(data):
  return data / 16384.0 - 0.5

In [152]:
def scalar_to_interval(x, rad):
  return ip.Interval(x - rad, x + rad)
scalar_to_interval_vec = np.vectorize(scalar_to_interval)

In [153]:
def get_iqr(x):
  q1 = np.percentile(x, 25, axis=0)
  q3 = np.percentile(x, 75, axis=0)
  return q1, q3

In [154]:
def get_box_plot(x):
  q1 = np.percentile(x, 25, axis=0)
  q3 = np.percentile(x, 75, axis=0)
  iqr = q3 - q1
  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr
  return lower_bound, upper_bound

In [155]:
data_folder = Path('data')
files = sorted([data_folder / f for f in os.listdir(data_folder) if f.endswith('.bin')])
files

[PosixPath('data/-0.027_lvl_side_a_fast_data.bin'),
 PosixPath('data/-0.205_lvl_side_a_fast_data.bin'),
 PosixPath('data/-0.471_lvl_side_a_fast_data.bin'),
 PosixPath('data/-0.492_lvl_side_a_fast_data.bin'),
 PosixPath('data/0.061_lvl_side_a_fast_data.bin'),
 PosixPath('data/0.225_lvl_side_a_fast_data.bin'),
 PosixPath('data/0.43_lvl_side_a_fast_data.bin'),
 PosixPath('data/0_lvl_side_a_fast_data_last.bin')]

In [156]:
estimation_functions = [
  ('IQR', get_iqr),
  ('Box plot', get_box_plot),
]

In [157]:
rad = 2 ** -14

for estimation_name, estimation_f in estimation_functions:
  betas = []

  px_num = 1024 * 8

  for px_index in tqdm(range(px_num), desc=estimation_name):
    A = []
    b = []

    for file_path in files:
      x = float(file_path.name.split('_')[0])
      frames = read_bin_file_with_numpy(file_path)
      ys = []

      for frame in frames:
        px = frame.flatten()[px_index]
        ys.append(px)

      y = estimation_f(ys)

      A.append([
        [x - rad, x + rad],
        [1 - rad, 1 + rad],
      ])
      b.append(y)

    A = ip.Interval(A)
    b = ip.Interval(b)

    x = ip.linear.Rohn(A, b)
    betas.append(x)

betas

Box plot: 100%|██████████| 8192/8192 [01:00<00:00, 134.40it/s]


[Interval(['[12771.6, 14473.9]', '[7639.24, 8306.7]']),
 Interval(['[12787.9, 14485.8]', '[7683.57, 8309.66]']),
 Interval(['[12985.2, 14678.3]', '[7788.34, 8396.77]']),
 Interval(['[12974.5, 14702.9]', '[7753.24, 8365.73]']),
 Interval(['[13097.9, 14644.1]', '[7808.06, 8358.96]']),
 Interval(['[12875.2, 14644.2]', '[7666.11, 8322.37]']),
 Interval(['[12953.9, 14435.6]', '[7859.18, 8384.17]']),
 Interval(['[12785.1, 14422.9]', '[7790.58, 8414.84]']),
 Interval(['[12078.5, 13955.6]', '[7626.5, 8334.6]']),
 Interval(['[12101.8, 13958]', '[7677.35, 8367.36]']),
 Interval(['[12294.1, 14171.9]', '[7722.95, 8411.73]']),
 Interval(['[12391, 14080.5]', '[7737.91, 8361.2]']),
 Interval(['[12422.1, 14216.4]', '[7745.74, 8421.43]']),
 Interval(['[12248.2, 14159.2]', '[7600.88, 8334.18]']),
 Interval(['[12428.6, 13751.8]', '[7838.28, 8355.71]']),
 Interval(['[12147.8, 13935.1]', '[7730.68, 8403.53]']),
 Interval(['[12191.5, 13836.1]', '[7670.14, 8283.11]']),
 Interval(['[12141.1, 13894.2]', '[7699